In [1]:
%autoreload 2

In [2]:
import torch, safetensors, pathlib
import huggingface_hub as hf_hub, safetensors as st
from nnsight import LanguageModel
from sae import Sae
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

from transformer_lens import HookedTransformer
from sae_lens import SAE
from sae_lens.toolkit.pretrained_saes import get_gpt2_res_jb_saes

Triton not installed, using eager implementation of SAE decoder.


In [12]:
prompt = 'Logic is easy, but it is impossible to be logical to bitter end. It is considered truth if one'

model_id = 'meta-llama/Meta-Llama-3.1-8B-Instruct'

feature = 6689
scale = 20
max_new_tokens = 40
num_return_sequences = 10

if torch.backends.mps.is_available(): device = "mps"
elif torch.cuda.is_available(): device = 'cuda'
else: device = 'cpu'

In [5]:
model = AutoModelForCausalLM.from_pretrained(model_id, device_map='auto', low_cpu_mem_usage=True)
tokenizer = AutoTokenizer.from_pretrained(model_id)

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

Dropping extra args {'signed': False}


model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Some parameters are on the meta device because they were offloaded to the disk.


In [13]:
inputs = tokenizer(prompt, return_tensors="pt").to(device)

In [18]:
inputs

{'input_ids': tensor([[128000,  27849,    374,   4228,     11,    719,    433,    374,  12266,
            311,    387,  20406,    311,  26242,    842,     13,   1102,    374,
           6646,   8206,    422,    832]], device='mps:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]],
       device='mps:0')}

In [20]:
sae = Sae.load_from_hub('EleutherAI/sae-llama-3.1-8b-32x', hookpoint='layers.29.mlp', device=device)

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

Dropping extra args {'signed': False}


In [21]:
with torch.inference_mode():
  outputs = model(**inputs, output_hidden_states=True)
  latent_acts = [sae.encode(outputs.hidden_states[29])]

In [36]:
outputs.logits

tensor([[[-2.9511,  1.5943,  7.4130,  ...,  1.6402,  1.6402,  1.6402],
         [ 5.6696,  2.3355,  3.7905,  ..., -5.8401, -5.8403, -5.8404],
         [ 2.0789,  1.2966, -0.0946,  ..., -8.3018, -8.3017, -8.3015],
         ...,
         [ 7.9952,  5.1692,  2.5138,  ..., -5.6675, -5.6675, -5.6675],
         [ 2.3323,  2.0644, -0.5631,  ..., -8.2491, -8.2490, -8.2491],
         [ 4.1120,  4.4859,  3.9373,  ..., -5.1948, -5.1947, -5.1947]]],
       device='mps:0')

In [24]:
l = latent_acts[0]
l.top_indices, l.top_acts

(tensor([[[ 88290, 113519, 108890,  ...,    863, 129994, 121807],
          [ 96360,  21315,  58801,  ...,  20409,  84362,  29115],
          [ 47834,  44914,  14233,  ...,  54857,  18015,  95291],
          ...,
          [ 29238, 120395,  77180,  ...,  33662, 116106,  94123],
          [ 85340,  34458, 109670,  ..., 119287, 121843,  94770],
          [ 91277,  17100,   1518,  ...,  69911,   7855,  83043]]],
        device='mps:0'),
 tensor([[[14.8641, 14.7052, 14.5979,  ..., 12.7887, 12.7804, 12.7784],
          [ 1.2117,  1.1897,  1.1791,  ...,  0.9237,  0.9226,  0.9215],
          [ 1.5425,  1.4885,  1.3973,  ...,  1.0895,  1.0895,  1.0894],
          ...,
          [ 1.6315,  1.5188,  1.4945,  ...,  1.1529,  1.1526,  1.1523],
          [ 1.5072,  1.4983,  1.4491,  ...,  1.1689,  1.1662,  1.1647],
          [ 1.7338,  1.7059,  1.7001,  ...,  1.2518,  1.2516,  1.2503]]],
        device='mps:0'))

In [27]:
sae.W_dec[[6689]]  # feature index: logic

tensor([[ 0.0091, -0.0137, -0.0124,  ...,  0.0023,  0.0068, -0.0111]],
       device='mps:0', grad_fn=<IndexBackward0>)

In [33]:
v = sae.W_dec[[6689]]
v = v / v.norm()
v = v * scale

In [35]:
It would trouble me if my child identified as trans.This question is required.

torch.Size([1, 4096])